# XOX PnF charts
## Development Notes

Two methods for populating charts:

- **High and Low** price: whenever H and L are available
- **Last/Close** price: for illiquid assets (or indices). Also for intraday trading using realtime data

Scale types:

- Constant size boxes
- Variable size boxes

#### Chart creation process:

1. Loading data into dataframe - **get_price_data()**
2. getting the scale for the chart - **generate_scale()**
3. generate a table with rows representing the boxes drawn by a line of data. Each row has a *trend status* (either 1 or -1), a column low and a column high. Those values represent *boxes*, and must be included in the chart scale. This is handled by **get_pnf_ranges()**, which calls:
  *  **init_pnf()** first
  * then **update_pnf()**
  
```
get_chart() -> get_price_data()
               generate_scale()
               get_pnf_ranges() -> init_pnf()
                                   update_pnf()
               get_pnf_changes()
               get_pnf_columns() -> generate_column_range()
```

`get_chart()` should be replaced by the pnf_chart object

### Preparations

In [ ]:
import numpy as np
import pandas as pd

import sys
print(f"Python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

from xox_pnf.pnfplot import *

In [ ]:
# inputs:
data_file = 'MMO2.csv'
reversal_size = 3
box_size = 1

chart_params = {
    'data_file': data_file,
    'reversal_size': reversal_size,
    'box_size': box_size
}

Functions from pnfplot
---------------------------

The chart data processing is managed by get_chart():

In [ ]:
get_chart(chart_params)

Next, I break down the process using the functions employed by get_chart()

In [ ]:
price_data = get_price_data(data_file)

In [ ]:
price_data.info()

In [ ]:
low = price_data['Low'].min()
high = price_data['High'].max()
scale = generate_scale(low, high, box_size)

print("box size: ", box_size)
print("high, low: ", high, low)
print("nscale array: ", scale)

In [ ]:
high = 101.0
low = 0.0
box_size = 10
scale = generate_scale(low, high, box_size)

print("box size: ", box_size)
print("high, low: ", high, low)
print("nscale array: ", scale)

### The next two steps are managed by get_pnf_ranges()

In [ ]:
# initialise status and box arrays:
trend_status = np.zeros(len(price_data))
box_low = np.zeros(len(price_data))
box_high = np.zeros(len(price_data))

# Initialise the chart until a trend status (+/-1) is found
box_range = []
for index, row in enumerate(price_data.iterrows()):
    high = row[1]['High']
    low = row[1]['Low']
    close = row[1]['Close']
    status, box_range = init_pnf(scale, high, low, close, reversal_size, box_range)
    print("step ", index,"status " ,status,"box range", box_range)
    trend_status[index] = status
    box_low[index] = box_range.min()
    box_high[index] = box_range.max()
    if status != 0:
        break

In [ ]:
print(trend_status, box_range)

In [ ]:
# update_pnf()

### get_pnf_ranges() and get_pnf_changes()

In [ ]:
# This includes a call to init_pnf() as in the previous cell
pnf_data = get_pnf_ranges(price_data, scale, reversal_size)
pnf_data

In [ ]:
# Detects change of trend and adds it to time frame
# Note that the change column is 'shifted': it's True when a status change is detected on the next price line

pnf_data = get_pnf_changes(pnf_data)
pnf_data

### Generating the final column arrays

In [ ]:
# The final chart columns: pairs of trend status and array of boxes
columns = get_pnf_columns(pnf_data, scale)
columns

## Printing the PnF chart

In [ ]:
# Using the pnf_text() function to generate pnf as text

print(pnf_text(scale,columns))

In [ ]:
type(pnf_text(scale,columns))

## Using the pnf_chart class

In [ ]:
# class pnf_chart():
    
#     def __init__(self, chart_params):
#         data_file = chart_params['data_file']
        
#         self.reversal_size = chart_params['reversal_size']
#         self.box_size = chart_params['box_size']
#         self.price_data = get_price_data(data_file)
        
#         low = self.price_data['Low'].min()
#         high = self.price_data['High'].max()
#         self.scale = generate_scale(low, high, self.box_size)
        
#         pnf_data = get_pnf_ranges(self.price_data, self.scale, self.reversal_size)
#         self.pnf_data = get_pnf_changes(pnf_data)
        
#         self.columns = get_pnf_columns(pnf_data, self.scale)        
#         self.text = pnf_text(self.scale, self.columns)
        
#     def __str__(self):
#         return self.text
    
#     def __repr__(self):
#         return f'PnF chart of {data_file}'

In [1]:
import numpy as np
import pandas as pd

import sys
print(f"Python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

from xox_pnf.pnfplot import *

Python version: 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
pandas version: 1.4.2
numpy version: 1.22.3


In [2]:
# inputs:
data_file = 'MMO2.csv'
reversal_size = 5
box_size = 1

chart_params = {
    'data_file': data_file,
    'reversal_size': reversal_size,
    'box_size': box_size
}

pnf = PnfChart(chart_params)

In [ ]:
pnf.pnf_data

In [4]:
print(pnf)

93.............93
92..X..........92
91..XO.........91
90..XOX........90
89..XOXO.......89
88..XOXOX......88
87..XOXOXO.....87
86..XOXOXO.....86
85...O.OXO.....85
84.....OXO.....84
83.....OXO.....83
82.....OXO.....82
81.....OXO.....81
80.....O.O.....80
79.......O.....79
78.......O.....78
77.......O.....77
76.......O.....76
75.......OX....75
74.......OXO...74
73.......OXO...73
72.......OXO...72
71.......OXO...71
70.......OXO...70
69.......OXO...69
68.......O.OX..68
67.........OX..67
66.........OX..66
65.........OX..65
64.........OX..64
63.........OX..63
62.........OX..62
61.........OX..61
60.........OX..60
59.........OX..59
58.........O...58
57.............57


In [3]:
pnf.text

['93.............93',
 '92..X..........92',
 '91..XO.........91',
 '90..XOX........90',
 '89..XOXO.......89',
 '88..XOXOX......88',
 '87..XOXOXO.....87',
 '86..XOXOXO.....86',
 '85...O.OXO.....85',
 '84.....OXO.....84',
 '83.....OXO.....83',
 '82.....OXO.....82',
 '81.....OXO.....81',
 '80.....O.O.....80',
 '79.......O.....79',
 '78.......O.....78',
 '77.......O.....77',
 '76.......O.....76',
 '75.......OX....75',
 '74.......OXO...74',
 '73.......OXO...73',
 '72.......OXO...72',
 '71.......OXO...71',
 '70.......OXO...70',
 '69.......OXO...69',
 '68.......O.OX..68',
 '67.........OX..67',
 '66.........OX..66',
 '65.........OX..65',
 '64.........OX..64',
 '63.........OX..63',
 '62.........OX..62',
 '61.........OX..61',
 '60.........OX..60',
 '59.........OX..59',
 '58.........O...58',
 '57.............57']

In [ ]:
pnf

In [ ]:
pnf.first_day

In [ ]:
pnf.last_day